In [1]:
import imageio as io
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import tifffile
from tqdm.notebook import tqdm
import pathlib
from cellpose import models, core
import json
import pathlib
from pyometiff import OMETIFFReader



In [2]:
experiment_path = '/mnt/sata3/Xenium_Data_Storage_2/20241216_GFP/output-XETG00195__0041388__TMA00150__20241213__232337'
dapi_path = 'morphology.ome.tif'

In [4]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 6407680645
def read_dapi_image(path: str, downscale_factor: int = 2) -> np.ndarray:
    img_fpath = pathlib.Path(path)
    img = io.imread(img_fpath)
    return downscale_image(img, downscale_factor=downscale_factor)

def downscale_image(img: np.ndarray, downscale_factor: int = 2) -> np.ndarray:
    # Calculate the amount 
    # 
    # of padding needed for each axis
    pad_height = (downscale_factor - img.shape[0] % downscale_factor) % downscale_factor
    pad_width = (downscale_factor - img.shape[1] % downscale_factor) % downscale_factor

    # Pad the array with zeros
    img = np.pad(img, ((0, pad_height), (0, pad_width)), mode='constant')
    return img


In [5]:

img_fpath = pathlib.Path(os.path.join(experiment_path, dapi_path))

reader = OMETIFFReader(fpath=img_fpath)

img_array, metadata, xml_metadata = reader.read()

Key not found: 'NoneType' object has no attribute 'iter'
Key not found: 'NoneType' object has no attribute 'iter'


In [7]:
np.shape(img_array)
sub_img_array = img_array[:,30000:35000,30000:35000]
sub_img_array = np.max(sub_img_array, axis = 0)

In [8]:
cellpose_training_folder = '/mnt/sata4/cellpose_training'
total_path = os.path.join(cellpose_training_folder, os.path.basename(experiment_path))

In [9]:
try:
    os.makedirs(total_path)
except:
    print('Folder already exists')


Folder already exists


In [9]:

#Define the chunk size
chunk_size = (1000, 1000)

for i in tqdm(range(0, sub_img_array.shape[1], int(chunk_size[1]/2))):
    for j in range(0, sub_img_array.shape[0], int(chunk_size[0]/2)):
        # Define the coordinates for cropping
        left = i
        upper = j
        right = i + chunk_size[1]
        lower = j + chunk_size[0]

        # Crop the image chunk
        chunk = sub_img_array[upper:lower, left:right]

        if np.sum(chunk) > 0:
            io.imwrite(os.path.join(total_path, f'chunk_{i}_{j}.tiff'), chunk)

  0%|          | 0/108 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: '/mnt/sata4/cellpose_training/output-XETG00195__0041388__TMA00150__20241213__232337/chunk_0_2500.tiff'